In [136]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from Utility.Get_dataset_clean_function import download_data_from_blob
from collections import Counter
from sklearn.neighbors import KNeighborsRegressor

In [98]:
df = pd.read_csv('../Documentation/data_for_model_2018_dep_1.csv', sep=',')

In [99]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df = df.drop(df[df['valeur_fonciere']<100000].index)

In [100]:
df.reset_index(inplace=True, drop=True)

In [102]:
# Recuperation données pour construction modele ml

classe_liste_prefixe_voie = download_data_from_blob('ABREVIATION_VOIE.csv')
classe_liste_prefixe_voie = list(classe_liste_prefixe_voie['Abreviation voie retenue'].to_dict().values())
classe_liste_prefixe_voie = [i for i in classe_liste_prefixe_voie if i in df.columns.values]

classe_liste_code_culture = download_data_from_blob('CODE_CULTURE.csv')
classe_liste_code_culture = list(classe_liste_code_culture['Code_nature_culture'].to_dict().values())

classe_liste_code_culture_spe = download_data_from_blob('CODE_CULTURE_SPECIALE.csv')
classe_liste_code_culture_spe = list(classe_liste_code_culture_spe['CODE_CULTURE_SPECIALE'].to_dict().values())

# Multi PCA

In [103]:
pca_code_culture = PCA(n_components=10)
df_code_culture = df.loc[:, classe_liste_code_culture]
df.drop(classe_liste_code_culture, axis=1, inplace=True)

columns_name_code_culture = ['code_culture_' + str(i) for i in range(10)]

code_culture_reduct = pca_code_culture.fit_transform(df_code_culture)
code_culture_reduct_df = pd.DataFrame(data=code_culture_reduct, columns=columns_name_code_culture)

In [104]:
code_culture_reduct_df.shape

(8791, 10)

In [105]:
pca_prefixe_voie = PCA(n_components=10)
df_prefixe_voie = df.loc[:, classe_liste_prefixe_voie]
df.drop(classe_liste_prefixe_voie, axis=1, inplace=True)

columns_name_prefixe = ['code_prefixe_' + str(i) for i in range(10)]

prefixe_reduct = pca_prefixe_voie.fit_transform(df_prefixe_voie)
prefixe_reduct_df = pd.DataFrame(data=prefixe_reduct, columns=columns_name_prefixe)

In [106]:
prefixe_reduct_df.shape

(8791, 10)

In [107]:
pca_code_culture_spe = PCA(n_components=10)
classe_liste_code_culture_spe = [i for i in classe_liste_code_culture_spe if i in df.columns.values]
df_code_culture_spe = df.loc[:, classe_liste_code_culture_spe]
df.drop(classe_liste_code_culture_spe, axis=1, inplace=True)

columns_name_code_culture_spe = ['code_culture_spe_' + str(i) for i in range(10)]

code_culture_spe_reduct = pca_code_culture_spe.fit_transform(df_code_culture_spe)
code_culture_spe_reduct_df = pd.DataFrame(data=code_culture_spe_reduct, columns=columns_name_code_culture_spe)

In [108]:
code_culture_spe_reduct_df.shape

(8791, 10)

In [85]:
pd.concat([df, prefixe_reduct_df, code_culture_reduct_df, code_culture_spe_reduct_df], axis=1).isna().sum()

date_mutation                               3794
code_postal                                 3794
code_commune                                3794
code_departement                            3794
nombre_lots                                 3794
surface_reelle_bati                         3794
nombre_pieces_principales                   3794
surface_terrain                             3794
valeur_fonciere                             3794
longitude                                   3794
latitude                                    3794
Appartement                                 3794
Dépendance                                  3794
Maison                                      3794
Autre                                       3794
Local industriel. commercial ou assimilé    3794
OTHER                                       3794
Vente                                       3794
Vente en l'état futur d'achèvement          3794
Echange                                     3794
Vente terrain à bâti

In [109]:
df = df.join(prefixe_reduct_df.join(code_culture_reduct_df.join(code_culture_spe_reduct_df)))

In [112]:
df.isna().sum()

date_mutation                               0
code_postal                                 0
code_commune                                0
code_departement                            0
nombre_lots                                 0
surface_reelle_bati                         0
nombre_pieces_principales                   0
surface_terrain                             0
valeur_fonciere                             0
longitude                                   0
latitude                                    0
Appartement                                 0
Dépendance                                  0
Maison                                      0
Autre                                       0
Local industriel. commercial ou assimilé    0
OTHER                                       0
Vente                                       0
Vente en l'état futur d'achèvement          0
Echange                                     0
Vente terrain à bâtir                       0
Adjudication                      

In [113]:
X = df.drop(labels='valeur_fonciere', axis=1)
Y = df['valeur_fonciere']

In [114]:
del df, prefixe_reduct_df, code_culture_reduct_df, code_culture_spe_reduct_df, pca_code_culture_spe,pca_prefixe_voie,pca_code_culture,code_culture_spe_reduct,prefixe_reduct, code_culture_reduct

In [115]:
Y_100 = (Y / 100).__round__()
Y_10000 = (Y_100 / 100).__round__()

In [131]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_10000, train_size=0.7)

# KNN

In [137]:
K = KNeighborsRegressor(n_neighbors=20)

In [138]:
K.fit(X_train,Y_train)

KNeighborsRegressor(n_neighbors=20)

In [139]:
prediction = K.predict(X_test)

# Logistic regression

In [133]:
LG = LogisticRegression()
LG.fit(X_train,Y_train)

LogisticRegression()

In [134]:
prediction = LG.predict(X_test)

# Random Forest

In [29]:
model = RandomForestClassifier(max_depth=50)

In [31]:
model.fit(X_train,Y_train)

MemoryError: could not allocate 358875136 bytes

In [ ]:
prediction = model.predict(X_test)

# Prediction calcul

In [135]:
Counter(prediction)

Counter({16.0: 2638})

In [125]:
Y_test[0:10]

7769     1975.0
8714     3570.0
1106     1958.0
2325     1000.0
6961     2950.0
623      1530.0
4203     2000.0
3826    21250.0
7042     2420.0
4809     1950.0
Name: valeur_fonciere, dtype: float64

In [140]:
c = 0
for x, y in zip(Y_test, prediction):
    if x == y:
        c += 1

BOW_ACCURACY = c / Y_test.shape[0]

print('Accuracy:' + str(BOW_ACCURACY))

Accuracy:0.0011372251705837756
